In [1]:
import pickle
import pandas as pd
import numpy as np
import pytds
import psycopg2
import math
import joblib
import os
from boltons import statsutils
from pdpbox import pdp, get_dataset, info_plots
import seaborn as sns
from statsmodels.robust.scale import huber, Huber
from datetime import datetime, timedelta
from copy import deepcopy
from numpy import mean, median
import pandas_profiling as pp
import gc
import pyodbc
from numpy import mean, median
import warnings
import itertools
warnings.filterwarnings('ignore')
from statsmodels.stats.weightstats import zconfint
from statsmodels.stats.proportion import binom_test
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
%matplotlib inline

In [2]:
data_folder = '../'

In [3]:
train_transaction = pd.read_csv(data_folder+'train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv(data_folder+'test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv(data_folder+'train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv(data_folder+'test_identity.csv', index_col='TransactionID')

In [4]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

In [6]:
# del train_transaction
# del test_transaction
# del train_identity
# del test_identity

In [7]:
mean_amt = pd.concat([train[['TransactionAmt']],test[['TransactionAmt']]])['TransactionAmt'].mean()

train['is_bigger_mean'] = (train['TransactionAmt']>mean_amt).astype(int)
test['is_bigger_mean'] = (test['TransactionAmt']>mean_amt).astype(int)

In [32]:
tmp = pd.crosstab(train['is_bigger_median'], train['isFraud'], normalize='index')*100

In [33]:
tmp

isFraud,0,1
is_bigger_median,,
0,96.624606,3.375394
1,96.378369,3.621631


In [35]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 'scranton.edu': 'other', 
          'optonline.net': 'other', 'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo', 
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink', 
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other', 'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo', 
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other', 'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft', 
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 
          'netzero.com': 'other', 'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other', 'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']

In [123]:
bin_to_num_dict = {'google':1, 'microsoft':2, 'other':3, 'yahoo':4, 'None':0}
suf_to_num_dict = {'com':1, 'nan':2, 'us':3}

In [140]:
def is_same_email_bin(P_email, R_email):
    P_email = str(P_email)
    R_email = str(R_email)
    if (R_email=='nan' and P_email=='nan'):
        return 1
    elif (R_email=='nan'):
        return 2
    elif (P_email=='nan'):
        return 3
    elif (P_email==R_email):
        return 4
    else:
        return 0

In [141]:
for c in ['P_emaildomain', 'R_emaildomain']:
    train[c + '_bin'] = train[c].map(emails).fillna('nan')
    test[c + '_bin'] = test[c].map(emails).fillna('nan')
    
    train[c + '_suffix'] = train[c].map(lambda x: str(x).split('.')[-1])
    test[c + '_suffix'] = test[c].map(lambda x: str(x).split('.')[-1])
    
    train[c + '_suffix'] = train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test[c + '_suffix'] = test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

In [142]:
train['is_same_email_bin'] = train[['P_emaildomain_bin', 'R_emaildomain_bin']].apply(lambda r: is_same_email_bin(r['P_emaildomain_bin'],r['R_emaildomain_bin']), axis=1)
test['is_same_email_bin'] = test[['P_emaildomain_bin', 'R_emaildomain_bin']].apply(lambda r: is_same_email_bin(r['P_emaildomain_bin'],r['R_emaildomain_bin']), axis=1)

train['is_same_email_suf'] = train[['P_emaildomain_suffix', 'R_emaildomain_suffix']].apply(lambda r: is_same_email_bin(r['P_emaildomain_suffix'],r['R_emaildomain_suffix']), axis=1)
test['is_same_email_suf'] = test[['P_emaildomain_suffix', 'R_emaildomain_suffix']].apply(lambda r: is_same_email_bin(r['P_emaildomain_suffix'],r['R_emaildomain_suffix']), axis=1)

In [124]:
for c in ['P_emaildomain', 'R_emaildomain']:
    train[c + '_bin'] = train[c + '_bin'].map(bin_to_num_dict).fillna(5)
    test[c + '_bin'] = test[c + '_bin'].map(bin_to_num_dict).fillna(5)
    
    train[c + '_suffix'] = train[c + '_suffix'].map(suf_to_num_dict).fillna(4)
    test[c + '_suffix'] = test[c + '_suffix'].map(suf_to_num_dict).fillna(4)

In [85]:
train.groupby('P_emaildomain_bin', as_index=False)['isFraud'].mean().sort_values('isFraud')

,P_emaildomain_bin,isFraud
5,5.0,0.021251
4,4.0,0.022052
3,3.0,0.025914
0,0.0,0.029538
1,1.0,0.043496
2,2.0,0.053298


In [86]:
def plot_categorical_feature(df, feature, target):
    temp_bad = df.loc[df[target]==1]
    temp_bad = temp_bad.groupby(feature, as_index=False)[target].count()
    temp_good = df.loc[df[target]==0]
    temp_good = temp_good.groupby(feature, as_index=False)[target].count()
    temp = temp_good.merge(temp_bad, on=feature, suffixes=('_g', '_b'))
    temp['perc'] = round(100*temp[target+'_b']/(temp[target+'_b']+temp[target+'_g']),2)
    trace_1 = go.Bar(x = temp_good[feature], y=temp_good[target], name='Хорошие')
    trace_2 = go.Bar(x = temp_bad[feature], y=temp_bad[target], name='Плохие', text=temp.perc, textposition = 'auto')
    layout = go.Layout(barmode='stack')
    fig = go.Figure(data=[trace_1, trace_2], layout=layout)
    iplot(fig)

In [144]:
plot_categorical_feature(train, 'is_same_email_bin', 'isFraud')

In [75]:
train.R_emaildomain_bin.value_counts()/len(train)<0.02

None           False
google         False
microsoft      False
other          False
yahoo          False
aol             True
apple           True
att             True
spectrum        True
centurylink     True
Name: R_emaildomain_bin, dtype: bool

In [139]:
train[train.R_emaildomain_suffix=='nan'].iloc[0]['R_emaildomain_suffix']

AttributeError: 'str' object has no attribute 'dtype'